In [ ]:
import pygame
import heapq
from math import sqrt
import random

# Pygame ve font sistemini başlat
pygame.init()
pygame.font.init()

# Ekran boyutları
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
CELL_SIZE = 40

# Renkler
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
GOLD = (255, 215, 0)
GREEN = (0, 255, 0)

# Ekran oluşturma
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Pacman with Multiple Search Algorithms")

# Izgara boyutları
GRID_WIDTH = SCREEN_WIDTH // CELL_SIZE
GRID_HEIGHT = SCREEN_HEIGHT // CELL_SIZE

# Labirent oluşturma
def create_maze():
    maze = [[0 for x in range(GRID_WIDTH)] for y in range(GRID_HEIGHT)]
    
    # Dış duvarlar
    for x in range(GRID_WIDTH):
        maze[0][x] = 1
        maze[GRID_HEIGHT-1][x] = 1
    for y in range(GRID_HEIGHT):
        maze[y][0] = 1
        maze[y][GRID_WIDTH-1] = 1

    # İç duvarlar
    for x in range(3, 8):
        maze[4][x] = 1
        maze[GRID_HEIGHT-5][x] = 1
    
    for x in range(GRID_WIDTH-8, GRID_WIDTH-3):
        maze[4][x] = 1
        maze[GRID_HEIGHT-5][x] = 1

    for y in range(3, 8):
        maze[y][4] = 1
        maze[y][GRID_WIDTH-5] = 1

    return maze

maze = create_maze()

class Character:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
        self.path = []

    def draw(self):
        pygame.draw.circle(screen, self.color, 
                         (self.x * CELL_SIZE + CELL_SIZE//2, 
                          self.y * CELL_SIZE + CELL_SIZE//2), 
                          CELL_SIZE//2 - 5)

class Coin:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self):
        pygame.draw.circle(screen, GOLD, 
                         (self.x * CELL_SIZE + CELL_SIZE//2, 
                          self.y * CELL_SIZE + CELL_SIZE//2), 
                          CELL_SIZE//4)

class Button:
    def __init__(self, x, y, width, height, text, color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.font = pygame.font.Font(None, 36)
        self.is_selected = False

    def draw(self, screen):
        # Buton arkaplanı
        pygame.draw.rect(screen, self.color, self.rect)
        # Buton çerçevesi
        pygame.draw.rect(screen, WHITE, self.rect, 2)
        
        # Seçili durum için arka plan rengi
        if self.is_selected:
            highlight_color = (min(self.color[0] + 50, 255),
                             min(self.color[1] + 50, 255),
                             min(self.color[2] + 50, 255))
            pygame.draw.rect(screen, highlight_color, self.rect)

        # Yazıyı siyah renkte render et
        text_surface = self.font.render(self.text, True, BLACK)
        text_rect = text_surface.get_rect(center=self.rect.center)
        
        # Yazı arkaplanı için beyaz bir dikdörtgen
        padding = 4
        background_rect = pygame.Rect(text_rect.x - padding,
                                    text_rect.y - padding,
                                    text_rect.width + 2*padding,
                                    text_rect.height + 2*padding)
        pygame.draw.rect(screen, WHITE, background_rect)
        
        # Yazıyı çiz
        screen.blit(text_surface, text_rect)

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

def get_neighbors(pos):
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_x, new_y = pos[0] + dx, pos[1] + dy
        if (0 <= new_x < GRID_WIDTH and 
            0 <= new_y < GRID_HEIGHT and 
            maze[new_y][new_x] == 0):
            neighbors.append((new_x, new_y))
    return neighbors

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(start, goal):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        current = heapq.heappop(frontier)[1]

        if current == goal:
            break

        for next_pos in get_neighbors(current):
            new_cost = cost_so_far[current] + 1
            if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                cost_so_far[next_pos] = new_cost
                priority = new_cost + heuristic(goal, next_pos)
                heapq.heappush(frontier, (priority, next_pos))
                came_from[next_pos] = current

    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = came_from.get(current)
    path.reverse()
    return path if path and path[0] == start else []

def bfs(start, goal):
    frontier = [(start, [start])]
    visited = {start}
    
    while frontier:
        current, path = frontier.pop(0)
        
        if current == goal:
            return path
            
        for next_pos in get_neighbors(current):
            if next_pos not in visited:
                visited.add(next_pos)
                new_path = path + [next_pos]
                frontier.append((next_pos, new_path))
    return []

def dfs(start, goal):
    frontier = [(start, [start])]
    visited = {start}
    
    while frontier:
        current, path = frontier.pop()
        
        if current == goal:
            return path
            
        for next_pos in get_neighbors(current):
            if next_pos not in visited:
                visited.add(next_pos)
                new_path = path + [next_pos]
                frontier.append((next_pos, new_path))
    return []

def evaluate_state(pacman_pos, ghost_positions, coin_positions):
    """
    Oyun durumunu değerlendir.
    Yüksek skor iyi durum (Pacman için), düşük skor kötü durum.
    """
    if not coin_positions:  # Tüm coinler toplandıysa
        return 1000
        
    # En yakın ghost'a olan mesafe
    min_ghost_dist = float('inf')
    for ghost_pos in ghost_positions:
        dist = heuristic(pacman_pos, ghost_pos)
        min_ghost_dist = min(min_ghost_dist, dist)
    
    # En yakın coin'e olan mesafe
    min_coin_dist = float('inf')
    for coin_pos in coin_positions:
        dist = heuristic(pacman_pos, (coin_pos.x, coin_pos.y))
        min_coin_dist = min(min_coin_dist, dist)
    
    # Değerlendirme skoru
    score = 0
    score += min_ghost_dist * 2  # Ghost'lardan uzak durma önemli
    score -= min_coin_dist      # Coin'lere yakın olma önemli
    
    return score

def get_possible_moves(pos):
    """Geçerli hamleleri döndür"""
    moves = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_x, new_y = pos[0] + dx, pos[1] + dy
        if (0 <= new_x < GRID_WIDTH and 
            0 <= new_y < GRID_HEIGHT and 
            maze[new_y][new_x] == 0):
            moves.append((new_x, new_y))
    return moves

def minimax_alpha_beta(pacman_pos, ghost_positions, coin_positions, depth, alpha, beta, maximizing_player):
    """
    Minimax algoritması Alpha-Beta Pruning ile
    """
    if depth == 0:
        return evaluate_state(pacman_pos, ghost_positions, coin_positions), None
        
    if maximizing_player:  # Pacman'in sırası
        max_eval = float('-inf')
        best_move = None
        for move in get_possible_moves(pacman_pos):
            eval_score, _ = minimax_alpha_beta(move, ghost_positions, coin_positions, 
                                            depth - 1, alpha, beta, False)
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, best_move
        
    else:  # Ghost'ların sırası
        min_eval = float('inf')
        best_move = None
        # Her ghost için olası hamleleri değerlendir
        for ghost_idx, ghost_pos in enumerate(ghost_positions):
            for move in get_possible_moves(ghost_pos):
                new_ghost_positions = list(ghost_positions)
                new_ghost_positions[ghost_idx] = move
                eval_score, _ = minimax_alpha_beta(pacman_pos, new_ghost_positions, 
                                                coin_positions, depth - 1, alpha, beta, True)
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
                beta = min(beta, eval_score)
                if beta <= alpha:
                    break
        return min_eval, best_move

def find_path_minimax(pacman, ghosts, coins):
    """
    Minimax kullanarak Pacman için en iyi hamleyi bul
    """
    pacman_pos = (pacman.x, pacman.y)
    ghost_positions = [(ghost.x, ghost.y) for ghost in ghosts]
    coin_positions = coins
    
    _, best_move = minimax_alpha_beta(pacman_pos, ghost_positions, coin_positions, 
                                    depth=3, alpha=float('-inf'), beta=float('inf'), 
                                    maximizing_player=True)
    
    if best_move:
        return [pacman_pos, best_move]
    return []

def find_nearest_coin(pacman, coins):
    if not coins:
        return None
    
    nearest_coin = None
    min_distance = float('inf')
    
    for coin in coins:
        distance = heuristic((pacman.x, pacman.y), (coin.x, coin.y))
        if distance < min_distance:
            min_distance = distance
            nearest_coin = coin
            
    return nearest_coin

def generate_valid_coins(num_coins):
    coins = []
    while len(coins) < num_coins:
        x = random.randint(1, GRID_WIDTH-2)
        y = random.randint(1, GRID_HEIGHT-2)
        if (maze[y][x] == 0 and 
            not any(coin.x == x and coin.y == y for coin in coins) and
            not (x == 1 and y == 1)):
            coins.append(Coin(x, y))
    return coins

def draw_maze(screen, maze):
    for y in range(GRID_HEIGHT):
        for x in range(GRID_WIDTH):
            if maze[y][x] == 1:
                pygame.draw.rect(screen, BLUE, 
                               (x * CELL_SIZE, y * CELL_SIZE, 
                                CELL_SIZE, CELL_SIZE))

class GameState:
    MENU = "MENU"
    PLAYING = "PLAYING"
    GAME_OVER = "GAME_OVER"
    GAME_WON = "GAME_WON"

class Game:
    def __init__(self):
        self.state = GameState.MENU
        self.pacman_algorithm = "A*"
        self.ghost_algorithm = "A*"
        self.score = 0
        self.num_ghosts = 2  # Ghost sayısı
        self.init_game()

    def init_game(self):
        self.pacman = Character(1, 1, YELLOW)
        
        # Birden fazla ghost oluştur
        self.ghosts = []
        ghost_positions = [
            (GRID_WIDTH-2, GRID_HEIGHT-2),  # Sağ alt
            (GRID_WIDTH-2, 1),              # Sağ üst
            (1, GRID_HEIGHT-2)              # Sol alt
        ]
        for x, y in ghost_positions[:self.num_ghosts]:
            self.ghosts.append(Character(x, y, RED))
            
        self.coins = generate_valid_coins(15)
        self.score = 0

    def create_menu_buttons(self):
        button_width = 150
        button_height = 50
        button_spacing = 20
        start_x = (SCREEN_WIDTH - (3 * button_width + 2 * button_spacing)) // 2
        
        self.buttons = {
            'pacman_astar': Button(start_x, 100, button_width, button_height, "Pacman: A*", (100, 200, 100)),
            'pacman_bfs': Button(start_x + button_width + button_spacing, 100, button_width, button_height, "Pacman: BFS", (100, 200, 100)),
            'pacman_dfs': Button(start_x + 2 * (button_width + button_spacing), 100, button_width, button_height, "Pacman: DFS", (100, 200, 100)),
            
            'ghost_astar': Button(start_x, 180, button_width, button_height, "Ghost: A*", (200, 100, 100)),
            'ghost_bfs': Button(start_x + button_width + button_spacing, 180, button_width, button_height, "Ghost: BFS", (200, 100, 100)),
            'ghost_dfs': Button(start_x + 2 * (button_width + button_spacing), 180, button_width, button_height, "Ghost: DFS", (200, 100, 100)),
            
            'start': Button(SCREEN_WIDTH//2 - button_width//2, 260, button_width, button_height, "Start", GREEN)
        }

    def find_path(self, start, goal, algorithm):
        if algorithm == "A*":
            return a_star(start, goal)
        elif algorithm == "BFS":
            return bfs(start, goal)
        elif algorithm == "DFS":
            return dfs(start, goal)
        return []

    def run(self):
        self.create_menu_buttons()
        running = True
        clock = pygame.time.Clock()

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()
                    
                    if self.state == GameState.MENU:
                        for key, button in self.buttons.items():
                            if button.is_clicked(mouse_pos):
                                if key == 'pacman_astar':
                                    self.pacman_algorithm = "A*"
                                elif key == 'pacman_bfs':
                                    self.pacman_algorithm = "BFS"
                                elif key == 'pacman_dfs':
                                    self.pacman_algorithm = "DFS"
                                elif key == 'ghost_astar':
                                    self.ghost_algorithm = "A*"
                                elif key == 'ghost_bfs':
                                    self.ghost_algorithm = "BFS"
                                elif key == 'ghost_dfs':
                                    self.ghost_algorithm = "DFS"
                                elif key == 'start':
                                    self.state = GameState.PLAYING
                                    self.init_game()

                    elif self.state in [GameState.GAME_OVER, GameState.GAME_WON]:
                        if self.buttons['start'].is_clicked(mouse_pos):
                            self.state = GameState.MENU

            screen.fill(BLACK)

            if self.state == GameState.MENU:
                # Başlık
                title_font = pygame.font.Font(None, 48)
                title_text = "Pacman with Search Algorithms"
                title_surface = title_font.render(title_text, True, WHITE)
                title_rect = title_surface.get_rect(center=(SCREEN_WIDTH//2, 40))
                screen.blit(title_surface, title_rect)

                # Seçili algoritmaları göster
                selected_text = f"Selected: Pacman-{self.pacman_algorithm}, Ghost-{self.ghost_algorithm}"
                selected_font = pygame.font.Font(None, 36)
                selected_surface = selected_font.render(selected_text, True, GOLD)
                selected_rect = selected_surface.get_rect(center=(SCREEN_WIDTH//2, 340))
                screen.blit(selected_surface, selected_rect)

                # Butonları çiz
                for key, button in self.buttons.items():
                    if (key == f'pacman_{self.pacman_algorithm.lower()}' or 
                        key == f'ghost_{self.ghost_algorithm.lower()}'):
                        button.is_selected = True
                    else:
                        button.is_selected = False
                    button.draw(screen)

            elif self.state == GameState.PLAYING:
                if not self.pacman.path:
                    nearest_coin = find_nearest_coin(self.pacman, self.coins)
                    if nearest_coin:
                        self.pacman.path = self.find_path(
                            (self.pacman.x, self.pacman.y),
                            (nearest_coin.x, nearest_coin.y),
                            self.pacman_algorithm
                        )
                    elif not self.coins:
                        self.state = GameState.GAME_WON

                if self.pacman.path and len(self.pacman.path) > 1:
                    next_pos = self.pacman.path[1]
                    self.pacman.x, self.pacman.y = next_pos
                    self.pacman.path = self.pacman.path[1:]

                    for coin in self.coins[:]:
                        if self.pacman.x == coin.x and self.pacman.y == coin.y:
                            self.coins.remove(coin)
                            self.score += 1
                            self.pacman.path = []
                            break

                # Tüm ghostların hareketi
                for ghost in self.ghosts:
                    ghost_path = self.find_path(
                        (ghost.x, ghost.y),
                        (self.pacman.x, self.pacman.y),
                        self.ghost_algorithm
                    )
                    if ghost_path and len(ghost_path) > 1:
                        next_pos = ghost_path[1]
                        ghost.x, ghost.y = next_pos

                    # Ghost Pacman'i yakaladı mı kontrolü
                    if ghost.x == self.pacman.x and ghost.y == self.pacman.y:
                        self.state = GameState.GAME_OVER
                        break

                draw_maze(screen, maze)
                for coin in self.coins:
                    coin.draw()
                self.pacman.draw()
                for ghost in self.ghosts:
                    ghost.draw()

                # Skor gösterimi
                score_text = f"Score: {self.score}"
                score_surface = pygame.font.Font(None, 36).render(score_text, True, WHITE)
                screen.blit(score_surface, (10, 10))

            elif self.state == GameState.GAME_OVER:
                text = f"GAME OVER! Score: {self.score}"
                text_surface = pygame.font.Font(None, 74).render(text, True, RED)
                text_rect = text_surface.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2))
                screen.blit(text_surface, text_rect)
                self.buttons['start'].text = "Restart"
                self.buttons['start'].draw(screen)

            elif self.state == GameState.GAME_WON:
                text = f"YOU WIN! Score: {self.score}"
                text_surface = pygame.font.Font(None, 74).render(text, True, WHITE)
                text_rect = text_surface.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2))
                screen.blit(text_surface, text_rect)
                self.buttons['start'].text = "Restart"
                self.buttons['start'].draw(screen)

            pygame.display.flip()
            clock.tick(5)

# Oyunu başlat
game = Game()
game.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

: 